### Before performing this, get the api key from kaggle.

In [ ]:
!pip install kaggle
from google.colab import files
from datetime import datetime
api_token = files.upload()

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/


In [ ]:
!kaggle competitions download -c 'malware-classification'

trainLabels.csv: Skipping, found more recently modified local copy (use --force to force download)
  0% 0.00/185k [00:00<?, ?B/s]
100% 185k/185k [00:00<00:00, 51.3MB/s]
100% 17.5G/17.5G [04:50<00:00, 54.5MB/s]
100% 17.5G/17.5G [04:50<00:00, 64.7MB/s]
  0% 0.00/4.06M [00:00<?, ?B/s]
100% 4.06M/4.06M [00:00<00:00, 117MB/s]
100% 17.8G/17.8G [05:16<00:00, 52.4MB/s]
100% 17.8G/17.8G [05:16<00:00, 60.2MB/s]


In [ ]:
# Keep only train.7z, trainLables files remove other files as shown below
!rm test.7z
!rm sampleSubmission.csv.zip
!rm dataSample.7z

In [ ]:
#
data=!7z l train.7z
print(len(data))
print(type(data))

21759
<class 'IPython.utils.text.SList'>


In [ ]:
byte_file_list=[]
asm_file_list=[]
for i in data:
  if (i.endswith("bytes")):
    byte_file_list.append(i)
  elif (i.endswith("asm")):
    asm_file_list.append(i)



In [ ]:
print('Number of Byte files',len(byte_file_list))

Number of Byte files 10868


In [ ]:
print('Number of ASM files',len(asm_file_list))

Number of ASM files 10868


In [ ]:
byte_file_list[0]

'2015-01-29 05:00:00 ....A      5256192               train/01azqd4InC7m9JpocGv5.bytes'

In [ ]:
# Removing extra character and to get only the file name
(byte_file_list[0].split()[-1]).replace('train/', '')

'01azqd4InC7m9JpocGv5.bytes'

In [ ]:
# Doing the same operation for all the files
byte_file_name=[]
for i in range(0,len(byte_file_list)):
  byte_file_name.append(byte_file_list[i].split()[-1].replace('train/', ''))

In [ ]:
byte_file_name[:3]


['01azqd4InC7m9JpocGv5.bytes',
 '01IsoiSMh5gxyDYTl4CB.bytes',
 '01jsnpXSAlgw6aPeDxrU.bytes']

In [ ]:
# Finding only the file names of ASM files and storing the file names in an array
asm_file_name=[]
for i in range(0,len(asm_file_list)):
  asm_file_name.append(asm_file_list[i].split()[-1].replace('train/', ''))

In [ ]:
asm_file_name[:3]

['01azqd4InC7m9JpocGv5.asm',
 '01IsoiSMh5gxyDYTl4CB.asm',
 '01jsnpXSAlgw6aPeDxrU.asm']

In [ ]:
import os
# Getting the current working directory
os.getcwd()

'/content'

In [ ]:
os.listdir()

['.config',
 'train.7z',
 'trainLabels.csv',
 '.ipynb_checkpoints',
 'kaggle.json',
 'asmbatch',
 'sample_data']

In [ ]:
# Downloading only a single byte file from train.7z
%%time
file_name=byte_file_name[0]
!7z e train.7z -o/content *$file_name -r


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 18810691091 bytes (18 GiB)

Extracting archive: train.7z
--
Path = train.7z
Type = 7z
Physical Size = 18810691091
Headers Size = 339764
Method = LZMA:24
Solid = +
Blocks = 94

  0%      4% . train/kQEbWRHa04gOYDqM1NJ6.asm                                       9% 1 . train/KqEgONxfHdP5lLaBIGQk.asm                                        13% 1 . train/KqEgONxfHdP5lLaBIGQk.asm                                        18% 1 . train/KqEgONxfHdP5lLaBIGQk.asm

In [ ]:
# Deleting the file
os.remove(file_name)

In [ ]:
!mkdir bytefiles

In [ ]:
#To download all the byte files from train.7z and below code will take almost 15mins.
start=datetime.now()

!7z e train.7z -o/content/bytefiles *.bytes -r

end=datetime.now()
difference=end-start
print('Time taken to extract all the byte files',difference)



7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 18810691091 bytes (18 GiB)

Extracting archive: train.7z
--
Path = train.7z
Type = 7z
Physical Size = 18810691091
Headers Size = 339764
Method = LZMA:24
Solid = +
Blocks = 94

  0%      0% . train/kQEbWRHa04gOYDqM1NJ6.asm                                       0% 1 . train/KqEgONxfHdP5lLaBIGQk.asm                                         0% 2 . train/kqiOdVbRQlB2s907GLMv.asm                                         0% 5 . train/kQsiVxDbAXt23wRWal57.asm

In [ ]:
print('Number of byte files extracted is',len(os.listdir('bytefiles/')))

Number of byte files extracted is 10868


In [ ]:
#https://www.thepythoncode.com/article/get-directory-size-in-bytes-using-python

def get_directory_size(directory):
    """Returns the `directory` size in bytes."""
    total = 0
    try:
        # print("[+] Getting the size of", directory)
        for entry in os.scandir(directory):
            if entry.is_file():
                # if it's a file, use stat() function
                total += entry.stat().st_size
            elif entry.is_dir():
                # if it's a directory, recursively call this function
                total += get_directory_size(entry.path)
    except NotADirectoryError:
        # if `directory` isn't a directory, get the file size then
        return os.path.getsize(directory)
    except PermissionError:
        # if for whatever reason we can't open the folder, return 0
        return 0
    return total

In [ ]:
def get_size_format(b, factor=1024, suffix="B"):
    """
    Scale bytes to its proper byte format
    e.g:
        1253656 => '1.20MB'
        1253656678 => '1.17GB'
    """
    for unit in ["", "K", "M", "G", "T", "P", "E", "Z"]:
        if b < factor:
            return f"{b:.2f}{unit}{suffix}"
        b /= factor
    return f"{b:.2f}Y{suffix}"

In [ ]:
get_size_format(get_directory_size("bytefiles/"))

'47.36GB'

In [ ]:
#Delete all the byte files
!rm -r -f bytefiles/

In [ ]:
!mkdir asmbatch

mkdir: cannot create directory ‘asmbatch’: File exists


In [ ]:
asm_file_name[:3]

['01azqd4InC7m9JpocGv5.asm',
 '01IsoiSMh5gxyDYTl4CB.asm',
 '01jsnpXSAlgw6aPeDxrU.asm']

In [ ]:
count=0
batch=0
time_taken=[]
start=datetime.now()
for file in asm_file_name:
  # start=datetime.now()
  !7z e train.7z -o/content/asmbatch *$file -r
  count+=1
  #Add your code here
  if(batch == 10):
    print('Batch 10 breaking')
    break
  if(count==10):
      count=0
      batch+=1
      !rm -r -f asmbatch/
      !mkdir asmbatch
      end=datetime.now()
      print('*'*50)
      print('end, start',end,start)
      difference=end-start
      start=datetime.now()
      time_taken.append('Time taken for the batch {} of ASM files is {}'.format(batch,difference))
      print('Time taken for the batch {} of ASM files is {}'.format(batch,difference))


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 18810691091 bytes (18 GiB)

Extracting archive: train.7z
--
Path = train.7z
Type = 7z
Physical Size = 18810691091
Headers Size = 339764
Method = LZMA:24
Solid = +
Blocks = 94

  0%     35% - train/01azqd4InC7m9JpocGv5.asm                                      78% - train/01azqd4InC7m9JpocGv5.asm                                     Everything is Ok

Size:       58961309
Compressed: 18810691091

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30G

In [ ]:
time_taken

['Time taken for the batch 1 of ASM files is 0:00:11.662088',
 'Time taken for the batch 2 of ASM files is 0:00:25.348033',
 'Time taken for the batch 3 of ASM files is 0:00:40.882263',
 'Time taken for the batch 4 of ASM files is 0:00:56.417840',
 'Time taken for the batch 5 of ASM files is 0:01:07.492606',
 'Time taken for the batch 6 of ASM files is 0:01:23.329963',
 'Time taken for the batch 7 of ASM files is 0:01:38.304537',
 'Time taken for the batch 8 of ASM files is 0:01:48.310862',
 'Time taken for the batch 9 of ASM files is 0:01:59.320909',
 'Time taken for the batch 10 of ASM files is 0:02:19.766148']